In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import requests
import json
import time
from tqdm.auto import tqdm
import math
import os, os.path
import geopandas as gpd
import numpy as np
import folium
from folium import Marker
import warnings 
from geopy.geocoders import Nominatim
import ast
import re
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
import gensim
import gensim.downloader

import matplotlib.pyplot as plt
from gensim.models.phrases import Phrases, Phraser
from gensim.models import KeyedVectors
import seaborn as sns

from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

import os
import urllib.request
from scipy import spatial
from sklearn.manifold import TSNE
from numpy import dot
from numpy.linalg import norm



In [10]:
df = pd.read_csv("geo_100000_200000.csv",low_memory=False, lineterminator='\n')

In [11]:
df.head()

,Unnamed: 0,id,name,description,number_of_employees,seo_description,annual_revenue_estimation,address,founding_year,languages_spoken_at_company,geo_lon,geo_lat,industries,keywords,revenue_currency,headquarter_location_geo_code_json,Latitude,Longitude,Full_address
0,100000,320351,Sendero,"No matter your path to success, let’s blaze th...",500,"No matter your path to success, let’s blaze th...",50000000.0,"750 North Saint Paul Street, 75201, Dallas, TX...",2004,"[""English""]",-96.799879,32.785905,"['Management Beratung', 'marketing', 'Personal...","['business management consulting', 'organizati...",NaN,NaN,NaN,NaN,NaN
1,100001,320352,The Immigrant Learning Center,The Immigrant Learning Center gives immigrants...,50,The Immigrant Learning Center,1000000.0,"442 Main Street, 02148, Malden, MA, United States",1992,NaN,-71.067409,42.427020,"['Wohltätigkeitsorganisationen (Non-Profit)', ...",['non-profits & non-profit services'],NaN,NaN,NaN,NaN,NaN
2,100002,320353,freebird inc,NaN,23,NaN,10000000.0,"Cambridge, Massachusetts, Vereinigte Staaten",NaN,"[""English""]",-71.160332,42.404384,"['Freizeit, Reisen und Tourismus', 'data_scien...","['online travel', 'information technology & se...",NaN,NaN,NaN,NaN,NaN
3,100003,320354,Grover Zampa Vineyards,Grover Zampa Vineyards: A formidable combinati...,200,NaN,50000000.0,"401, ARCADIA, 400021, Nashik, Maharashtra, India",1989,"[""English""]",72.775909,19.271634,"['Alkoholische Getränke und Spirituosen', 'sal...",['wine & spirits'],NaN,NaN,NaN,NaN,NaN
4,100004,320355,Vecdis,La adecuada sistematización de la información ...,25,NaN,10000000.0,"CALLE MARÍA BENÍTEZ, 55, 28224, Madrid, Commun...",2003,"[""Spanish""]",-3.803755,40.453722,"['Informationstechnologie', 'media_and_commmun...","['vigilancia tecnologica', 'gestion del conoci...",NaN,NaN,NaN,NaN,NaN


In [12]:
companies_isna_stats = df.isna().sum() / df.shape[0] * 100
companies_isna_stats

Unnamed: 0                              0.000
id                                      0.000
name                                    0.000
description                             2.469
number_of_employees                     0.022
seo_description                        39.185
annual_revenue_estimation               0.027
address                                 0.691
founding_year                           9.566
languages_spoken_at_company            14.858
geo_lon                                15.402
geo_lat                                15.402
industries                              0.138
keywords                                0.548
revenue_currency                      100.000
headquarter_location_geo_code_json    100.000
Latitude                               66.766
Longitude                              66.766
Full_address                           66.766
dtype: float64

In [9]:
path_glv_50 = 'glove.6B/glove.6B.50d.txt'
embed_glv_50 = get_embedding_dict(path_glv_50)
tqdm.pandas()

In [123]:
df["ind_embed"] = df["eng_industries"].progress_apply(lambda row :  get_row_embedding(row, get_description_embedding_max, embed_glv_50, 50 ))  


  0%|          | 0/2846000 [00:00<?, ?it/s]

In [124]:
df["kw_embed"] = df["eng_keywords"].progress_apply(lambda row :  get_row_embedding(row, get_description_embedding_avg, embed_glv_50, 50 ))  


  0%|          | 0/2846000 [00:00<?, ?it/s]

In [126]:
ind_embeddings = df["ind_embed"].values
kw_embeddings = df["kw_embed"].values
embeddings = []
for i in range(len(ind_embeddings)):
    try:
        embedding = ind_embeddings[i] + kw_embeddings[i]
    except:
        print(kw_embeddings[i])
        print(kw_)

TypeError: unsupported operand type(s) for +: 'float' and 'NoneType'

In [125]:
ind_embeddings

NameError: name 'ind_embeddings' is not defined

In [ ]:
df["embeddings"]

In [86]:
industries_emb = df["ind_embed"].values
keywords_emb = df["kw_embed"].values

In [116]:
industries_emb.shape

(2846000,)

In [20]:
df["tokens_industries"].values[0]

"['semiconductor', 'marketing', 'information', 'technology', 'product', 'management', 'legal', 'business', 'development', 'human', 'resources', 'operations', 'education', 'support', 'finance', 'media', 'commmunication', 'semiconductors', 'engineering', 'sales']"

In [96]:
embedd_list = []
for e in tqdm(list(industries_emb)):
    try:
        embedd_list.append(list(e))
    except:
        embedd_list.append(list(np.zeros(50)))

  0%|          | 0/2846000 [00:00<?, ?it/s]

In [105]:
kmeans = KMeans(n_clusters=20, random_state=0).fit(np.array(embedd_list))

In [106]:
kmeans = KMeans(n_clusters=20, random_state=0, verbose=True).fit(np.array(embedd_list))

Initialization complete
Iteration 0, inertia 5921472.511018308
Iteration 1, inertia 4276336.228772683
Iteration 2, inertia 4102486.2424195176
Iteration 3, inertia 4036086.4487242484
Iteration 4, inertia 4003852.6827981556
Iteration 5, inertia 3989269.82774539
Iteration 6, inertia 3982111.4418778173
Iteration 7, inertia 3978342.1674371553
Iteration 8, inertia 3977438.392885362
Iteration 9, inertia 3971036.2961184187
Iteration 10, inertia 3969546.4415690624
Iteration 11, inertia 3969350.7998410817
Iteration 12, inertia 3969314.1039630687
Converged at iteration 12: center shift 1.34863313984511e-05 within tolerance 1.5604552531249312e-05.
Initialization complete
Iteration 0, inertia 5374100.892183979
Iteration 1, inertia 4046211.5496991547
Iteration 2, inertia 3979798.405200597
Iteration 3, inertia 3948990.217014883
Iteration 4, inertia 3936655.5639018756
Iteration 5, inertia 3932143.998106585
Iteration 6, inertia 3930525.0635232623
Iteration 7, inertia 3928433.004078993
Iteration 8, iner

In [107]:
examples = give_examples_of_clusters(20, kmeans)

In [109]:
for i in examples[1]:
    print(df["tokens_keywords"].values[i])

['law', 'practice', 'legal', 'services']
nan
['political', 'organization']
['electrical', 'electronic', 'manufacturing']
['professional', 'training', 'coaching']
nan
['facilities', 'services']
['hematology', 'cytology', 'client', 'service', 'pathology', 'phlebotomy', 'clinical', 'chemistry', 'laboratory', 'medicine', 'patient', 'care', 'hospital', 'health', 'care']
nan
['finance']
nan
nan
['hospital', 'health', 'care']
['trainingen', 'sales', 'social', 'media', 'business', 'development', 'communication', 'personal', 'development', 'professional', 'training', 'coaching', 'consumer', 'internet', 'consumers', 'internet', 'information', 'technology', 'services', 'communications']
nan
['live', 'events', 'strategic', 'marketing', 'plan', 'trade', 'shows', 'websites', 'referral', 'marketing', 'brand', 'development', 'advertising', 'multichannel', 'direct', 'marketing', 'national', 'sales', 'meetings', 'ambassador', 'marketing', 'social', 'media', 'roi', 'modeling', 'sales', 'collateral', 'mar

In [97]:
neigh = NearestNeighbors(n_neighbors=2, radius=0.4)
#neigh.fit(X)
neigh.fit(np.array(embedd_list))

NearestNeighbors(n_neighbors=2, radius=0.4)

In [103]:
indx = 108004
neighbours = neigh.kneighbors([np.array(embedd_list)[indx]], 10, return_distance=False)
print(df["tokens_industries"].values[indx])


['public', 'relations', 'communication', 'consulting', 'media', 'commmunication', 'marketing']


In [104]:
for i in neighbours:
    print(df["tokens_industries"].values[i])

["['design', 'marketing', 'business', 'development', 'consulting']"
 "['real', 'estate', 'industry', 'consulting', 'marketing', 'real', 'estate', 'industry']"
 "['marketing', 'consulting', 'marketing', 'advertisement']"
 "['marketing', 'consulting', 'marketing', 'advertisement']"
 "['higher', 'education', 'marketing', 'consulting']"
 "['marketing', 'advertisement', 'consulting', 'marketing', 'marketing', 'advertisement']"
 "['informationstechnologie', 'marketing', 'business', 'development', 'consulting', 'management', 'consulting']"
 "['higher', 'education', 'state', 'administration', 'media', 'commmunication', 'information', 'technology', 'consulting', 'informationstechnologie', 'marketing', 'writing', 'journalism', 'human', 'resources']"
 "['marketing', 'consulting', 'business', 'development', 'marketing', 'advertisement']"
 "['management', 'consulting', 'marketing', 'consulting']"]


In [110]:
Xtsne = TSNE(n_components=2, verbose=True).fit_transform(np.array(embedd_list))
dftsne = pd.DataFrame(Xtsne)

/Users/ahmedewva/opt/anaconda3/envs/idp/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/ahmedewva/opt/anaconda3/envs/idp/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 2846000 samples in 0.116s...


KeyboardInterrupt: 

In [ ]:
dftsne['cluster'] = kmeans.labels_
dftsne.columns = ['x1','x2','cluster']
fig, ax = plt.subplots(1, figsize=(16,8))
sns.scatterplot(data=dftsne,x='x1',y='x2',hue='cluster',legend="full",  palette="deep",alpha=0.5,ax=ax)
ax.set_title('Visualized on TSNE 2D')

In [114]:
df["ind_embed"].values[0]

1.15910005569458